In [1]:
import cv2
import numpy as np
import dlib
import math
import os

In [3]:
camera=cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def allign(im):
    
  detections = detector(im, 1)
  flag=2
  for k,d in enumerate(detections): #For each detected face   
   flag=1
   shape = predictor(im, d)
   a=(shape.part(36).x,shape.part(36).y)
   b=(shape.part(45).x,shape.part(45).y)
   p1=[a,b]
  if flag==2:
   return np.asrray([0])      
  p2=[((int(0.3*200),66)),(int(0.7*200),66)]

  s60 = math.sin(60*math.pi/180);
  c60 = math.cos(60*math.pi/180);  
  inPts = np.copy(p1).tolist();
  outPts = np.copy(p2).tolist();
  xin = c60*(inPts[0][0] - inPts[1][0]) - s60*(inPts[0][1] - inPts[1][1]) + inPts[1][0];
  yin = s60*(inPts[0][0] - inPts[1][0]) + c60*(inPts[0][1] - inPts[1][1]) + inPts[1][1];
  inPts.append([np.int(xin), np.int(yin)]);
  xout = c60*(outPts[0][0] - outPts[1][0]) - s60*(outPts[0][1] - outPts[1][1]) + outPts[1][0];
  yout = s60*(outPts[0][0] - outPts[1][0]) + c60*(outPts[0][1] - outPts[1][1]) + outPts[1][1];
  outPts.append([np.int(xout), np.int(yout)]);
  tform = cv2.estimateRigidTransform(np.array([inPts]), np.array([outPts]), False);
  img2 = cv2.warpAffine(im, tform, (200,200));
  detections = detector(img2, 1)
  for k,d in enumerate(detections): 
   face=[[d.left(),d.right()],[d.top(),d.bottom()]]
  if shape.part(20).y>shape.part(25).y:
    top=shape.part(20).y
  else :
    top=shape.part(25).y

  img2_cropped=img2[top:face[1][1],face[0][0]:face[0][1]]


  img2_cropped=cv2.resize(img2_cropped,(200,200))
  return img2_cropped


def face_detect(gray):
    detections = detector(gray, 1)
    flag=0
    face=[1]
    for k,d in enumerate(detections):  
      flag=1
      
      face=gray[d.top():d.bottom(),d.left():d.right()]
    return face,flag 

def gamma(image,gamma):
    invGamma = 1.0 / gamma
    table1 = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    p=cv2.LUT(image, table1)
    return p

def dog(img):
    g1=cv2.GaussianBlur(img,(3,3),0)
    g2=cv2.GaussianBlur(img,(23,23),0)
    return g2-g1
i=0
while(1):
    
    ret,image=camera.read()
    if ret == False:
        continue
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    face,flag=face_detect(gray)
    if flag==1:
        face=cv2.resize(face,(200,200))
        face = cv2.equalizeHist(face)
        allign_face=allign(face)
        #if allign_face==np.asrray([0]):
         #continue
        gamma_face=gamma(allign_face,0.5)
        gauss_face=dog(gamma_face)
        
        cv2.imshow('img',allign_face)
    
        if cv2.waitKey(1)==ord('s'):
          print 'saved'  
          while os.path.exists(str(i)):
           i = i+1
          os.makedirs(str(i))
          os.chdir(str(i))
          for j in range(1,11):
           cv2.imwrite(str(j)+'.jpeg',gauss_face)     
           os.chdir('..')
    
    '''    
    if flag==0:
        cv2.imshow('img',gray)
    '''
    if cv2.waitKey(1)==ord('q'):
        break
     
    
        
camera.release()
cv2.destroyAllWindows()

In [5]:
def p():
    flag=1
    a=2
    return flag,a
a,b=p()
print a,b


1 2


In [4]:
import os
os.makedirs(str(0))
